# TensorFlow and Computer Vision - DIY

### Background:

You are now asked to solve another problem based on computer vision. Your task is to identify Sign Language Digits. You will again use the TensorFlow framework to solve the sign identification problem.

### About the dataset
You will use the Sign Language Digits Dataset as a sample dataset. Sign Language Digits Dataset distinguishes the sign language digits from 0–9. The following image shows a sample of the dataset.

![](img/sign-number.png)


This lab is designed based on the aws-samples available in the following [github repo](https://github.com/aws-samples/sagemaker-multi-model-endpoint-tensorflow-computer-vision).

**Note** To run the code cell, press **SHIFT + ENTER**. When the cell finishes running, the text to the left of the cell changes from **In [*]:** to **In [1]**.

### Setup

 #### Prerequisites 
 Update the Sagemaker to the latest version and install the TensorFlow framework 2.6.1.

In [ ]:
%%capture
%pip install sagemaker
%pip install tensorflow
%pip install matplotlib

In [ ]:
# Validate if the TensorFlow is installed correctly.
import tensorflow as tf
print(f"TensorFlow version:{tf.__version__}")

#### Imports 

In [ ]:
import os
import time
import boto3
import logging
import sagemaker
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import utils
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sagemaker.tensorflow.serving import TensorFlowModel
from tensorflow.keras.datasets import cifar10

from sagemaker import get_execution_role

#### Setup Logger

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'[Using TensorFlow version: {tf.__version__}]')
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

#### Seed for Reproducability

In [ ]:
SEED = 123
np.random.seed(SEED)
tf.random.set_seed(SEED)

#### Create Roles, Sessions and Data Locations

Rememeber to replace **\<Enter_Your_Bucket_Name\>** with the bucket name.

In [ ]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.11.0'
BUCKET = '<Enter_Your_Bucket_Name>'
PREFIX = 'cv-models'

### Train Model - Sign Language Image Classification

#### a) Load Data

In [ ]:
train_path  = './data/sign_language/train'
validation_path  = './data/sign_language/validation'
test_path  = './data/sign_language/test'

#### b) Data Exploration

In [ ]:
%matplotlib inline
img = mpimg.imread(f'{train_path}/0/IMG_1118.JPG')
plt.imshow(img)

In [ ]:
img.shape

<p>ImageDataGenerator generates batches of tensor image data with real-time data augmentation. 
The data will be looped over (in batches).

In [ ]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, 
                                                         target_size=(224, 224), 
                                                         batch_size=10)

In [ ]:
images, labels = next(train_batches)

In [ ]:
images[0].shape

In [ ]:
# Create a figure with 4 subplots
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 4))

# Plot the first 4 images in the batch
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i].astype('int'))
    ax.axis('off')

plt.tight_layout()
plt.show()

Visualize random sample of images from `train_batches`

#### c) Copy local data to S3 for SageMaker training

In [ ]:
!aws s3 cp ./data/sign_language/ s3://{BUCKET}/{PREFIX}/sign_language/ --recursive

#### d) Define Model Architecture & create Training Script

In [ ]:
!pygmentize sign_language_train.py

#### f) Create a TensorFlow Estimator & fit the Model

Remember to enter **InstanceType** and **epochs**

In [ ]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/sign_language/train', 
                            distribution='ShardedByS3Key')
val_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/sign_language/validation', 
                          distribution='ShardedByS3Key')

In [ ]:
model_name = 'sign-language'
instance_type = '<Enter_Instance_Type>'
hyperparameters = {'epochs': <Enter_Epoch_Value>}

estimator_parameters = {'entry_point':'sign_language_train.py',
                        'instance_type': instance_type,
                        'instance_count': 1,
                        'hyperparameters': hyperparameters,
                        'model_dir': f'/opt/ml/model',
                        'role': role,
                        'output_path': f's3://{BUCKET}/{PREFIX}/sign_language/out',
                        'base_job_name': f'cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py39',
                        'script_mode': True}

estimator = TensorFlow(**estimator_parameters)

In [ ]:
estimator.fit({'train': train_input, 'val': val_input})

### Create an Real time Inference Endpoint

#### a) Copy Trained Models to a common S3 Prefix

In [ ]:
tf_model_sign = estimator.model_data
output = f's3://{BUCKET}/{PREFIX}/sign-language.tar.gz'

In [ ]:
!aws s3 cp {tf_model_sign} {output}

#### b) Essentials

In [ ]:
current_time = datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H-%M-%S')
current_time

**IMAGE_URI Format:** image_name='763104351884.dkr.ecr.\<region\>.amazonaws.com/\<framework\>-\<job type\>:\<framework version\>-\<cpu/gpu\>-\<python version\>-ubuntu20.04-v1'

Use following [link](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html) to use TensorFlow with the SageMaker Python SDK.

In [ ]:
IMAGE_URI = '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.6-gpu-py38-cu112-ubuntu20.04-v1'
model_data_prefix = f's3://{BUCKET}/{PREFIX}/'

#### c) Define a model using tf_model_sign

In [ ]:
tf_model_sign = TensorFlowModel(model_data=output, 
                          role=role, 
                          image_uri=IMAGE_URI)

#### d) Deploy a real time Inference Endpoint

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


instance_type = "ml.m5.xlarge"

predictor = tf_model_sign.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
)

### Test Real Time Inference Endpoint

#### a) Test Model-1 CIFAR Image Classification

In [ ]:
%matplotlib inline
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np

#### b) Test Sign Language Classification

In [ ]:
test_path  = './data/sign_language/test'
img = mpimg.imread(f'{test_path}/3/IMG_5114.JPG')
plt.imshow(img)

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [ ]:
data = path_to_tensor(f'{test_path}/3/IMG_5114.JPG')
payload = {'instances': data}

In [ ]:
y_pred = predictor.predict(data=payload)

In [ ]:
predicted_label = np.argmax(y_pred['predictions'])
print(f'Predicted Label: [{predicted_label}]')

In [ ]:
y_pred

In [ ]:
np.argmax(y_pred['predictions'])

## Cleanup (Optional)

Uncomment the below code block to delete the Sagemaker Endpoint.

In [ ]:
#predictor.delete_endpoint()